In [1]:
#ONLY RUN THIS FILE ONCE! get_all_objects() is slow... luckily I've saved the resulting dataframe as a .pkl

import json
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import nltk
from nltk.stem import PorterStemmer as ps
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
stemmer = ps()
from nltk.stem.cistem import Cistem
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

#display max rows
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
# Function to format the JSON objects correctly
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [8]:
#Function to take JSON objects and turn them into a DF across multiple pages
#This does all the pages of Unknown makers with 25 results per page. NOTE: This will take ~forever to run

def get_all_objects():
    data_list = []
    for x in range(0, 1392):
        parameters = {"search_api_fulltext": "Unknown",
                      "items_per_page": 25,
                      "page": x
                      }
        response = requests.get("https://risdmuseum.org/api/v1/collection", params=parameters).json()
        for item in response:
            data_list.append(item)

    df = pd.DataFrame()
    for each_dict in data_list:
        new_df = pd.json_normalize(each_dict)
        df = df.append(new_df)
    return df

In [9]:
##Getting all Maker Unknown objects from DB
object_df = get_all_objects()

In [10]:
##caching with pkl so we don't have to retrieve every time
object_df.to_pickle("./unknown_makers_full.pkl")

In [5]:
##A function to get total numbers for a particular individual category, i.e. culture, dating, etc. 
def get_counts(original_df, category_name, new_name):
    if(isinstance(original_df[category_name].iloc[0], list)):
        mega_list = []
        for row in original_df[category_name]:
            item_list = row
            for item in row: 
                mega_list.append(item)
                mega_list.sort()
        new_dict = {}
        for category in mega_list:
            if not category in new_dict.keys():
                new_dict[category] = 1
            else:
                new_dict[category] +=1
        counts_df = pd.DataFrame([new_dict]).melt()

    else:
        counts = original_df.groupby([category_name]).size()
        counts_df = counts.to_frame().reset_index(category_name)
    counts_df.columns = [new_name, 'Object Count']
    sorted_df = counts_df.sort_values('Object Count', ascending = False)
    return sorted_df

In [ ]:
##Get counts for the type of object
get_counts(object_df,'type', 'Object Type')

In [ ]:
#Get counts for each culture
get_counts(object_df,'culture', 'Culture Label')

In [ ]:
#Get counts for each date: Question for Denise- how best to categorize?
#Regex needed if I wanted to explore this further
get_counts(object_df,'dating', 'Approx. Date')

In [ ]:
#Get counts for medium type
get_counts(object_df,'medium', 'Medium Type')